**IMPORTING NON-STANDARD LIBRARIES**

*pyart* will be central to creating our model as it has a class *Radar* and abilities to read NEXRAD files as objects of the class.

Documentation: https://arm-doe.github.io/pyart/API/generated/pyart.core.Radar.html?highlight=radar#pyart.core.Radar

In [ ]:
import platform
mysystem = platform.system()

# Check for windows
if mysystem != 'Windows':
    !unzip "allscreenings"

In [ ]:
# install a pip package in the current jupyter kernel
import sys
!{sys.executable} -m pip install arm_pyart matplotlib
!{sys.executable} -m pip install cartopy boto3
!{sys.executable} -m pip install botocore
#!{sys.executable} -m pip install botocore==1.27.60
#!{sys.executable} -m pip install boto3

**GAME PLAN**

1. Iterate through all stations, months, days
2. Fetch NEXRAD files for each day and create arrays of this data.
4. Link dates in XLSX sheets with dates of NEXRAD files.
3. Process and train models on preprocessed files.

Here is in progress code that will complete step 1 and 2.

In [ ]:
import pyart as pa

import pandas as pd

import boto3
import botocore
from botocore import UNSIGNED
from botocore.client import Config

import cartopy.crs as ccrs
import matplotlib.pyplot as plt

debug = True

stations = ["KABR", "KIND", "KJKL", "KDHL"]
months = ["11", "12", "01", "02", "03"]
days = ["01", "02", "03", "04", "05", "06", "07", "08", "09"]
for day in range(10,32):
  days.append(day)

if debug:
  # less data to test
  stations = ["KIND"]
  months = ["11"]
  days = ["01"]

bucket_links = []

s3 = boto3.resource('s3', config = Config(signature_version = botocore.UNSIGNED, user_agent_extra = 'Resource'))
bucket = s3.Bucket('noaa-nexrad-level2')
for station in stations:
  for month in months:
    for day in days:
      for year in ['2019', '2020']:
        for obj in bucket.objects.filter(Prefix = f'{year}/{month}/{day}/{station}/{station}{year}'):
          if not ("MDM" in obj.key):
            bucket_links.append(f's3://noaa-nexrad-level2/{obj.key}')

KABR = []
KIND = []
KJKL = []
KDHL = []
aws_nexrad_level2_file = "s3://noaa-nexrad-level2/2022/03/22/KHGX/KHGX20220322_120125_V06"
radar2 = pa.io.read_nexrad_archive(aws_nexrad_level2_file)

for link in bucket_links:
  print(link)
  radar = pyart.io.read_nexrad_archive(link)
  if "KABR" in link:
    KABR.append(radar)
  elif "KIND" in link:
    KIND.append(radar)
  elif "KJKL" in link:
    KJKL.append(radar)
  else:
    KDHL.append(radar)
  if debug: 
    break

print(KABR)
print(KIND)
print(KJKL)
print(KDHL)


In [ ]:
import pyart

if pyart.__version__ == "1.7.0":
    if hasattr(pyart.graph, 'RadarMapDisplay'):
        print("Ready to go")
    else:
        print ("ISSUES\n\nMissing\Broken Basemap\n")
        pyart._debug_info()
else:
    print ("ISSUES\n\nWrong Py-ART version\n")
    pyart._debug_info()

For each day, there is a value associated with it: C (contaiminated) and NC (non-contaminated). In the data, this was evaluated by humans. Ideally, our model will be able to deduce this by itself. The following code fetches data from some example stations and returns the classification for all days recorded.

In [14]:
# Iterate through allscreening folder

# import required module
import os
import pandas
# assign directory
directory = 'allscreenings'
 
# iterate over files in
# that directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)

    sheet = pandas.read_excel(f)
    # checking if it is a file
    if os.path.isfile(f):
        print(sheet['STATUS'])

0      B
1      C
2      B
3      C
4      B
      ..
147    C
148    C
149    B
150    C
151    C
Name: STATUS, Length: 152, dtype: object
0       B
1       C
2      NB
3       C
4       B
       ..
147     C
148     C
149     B
150     C
151     C
Name: STATUS, Length: 152, dtype: object
0      C
1      C
2      C
3      C
4      C
      ..
178    C
179    C
180    C
181    C
182    C
Name: STATUS, Length: 183, dtype: object
0      C
1      C
2      C
3      C
4      C
      ..
147    C
148    C
149    C
150    C
151    C
Name: STATUS, Length: 152, dtype: object
